<a href="https://colab.research.google.com/github/MarijanMarkovic/Numerical-modelling-of-differential-equations/blob/main/StationaryNS_vj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install -y --no-install-recommends fenics
!rm -rf

In [ ]:
from fenics import*
from matplotlib import pyplot as plt

#1 Mesh Gen
nx, ny=20, 20
mesh=UnitSquareMesh(nx, ny, 'crossed')
degree=1

#2 Finite Element Space
V=VectorElement('CG', mesh.ufl_cell(), degree+1)
Q=FiniteElement('CG', mesh.ufl_cell(), degree)

X=FunctionSpace(mesh, MixedElement([V, Q]))

u_boundary=Expression(('near(x[1], 1) ? 1.0 : 0.0', '0'),degree=0)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc=[
    DirichletBC(X.sub(0), u_boundary, boundary),
    DirichletBC(X.sub(1), Constant(0.0), origin, 'pointwise')
]

#3 Problem Definiton

def solve_stationary_NS(X, advection, Re, f, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  a = (inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
  L = dot(f, v) * dx
  if advection:
    a += (dot(grad(u) * advection, v) + dot(grad(advection) * u, v)) * dx #NEWTONOVA METODA!!!
    L += dot(grad(advection) * advection, v) * dx                         #BRŽE KONVERGIRA
  
  x = Function(X)
  solve(a == L, x, bc)
  
  return x.split()

# 4. solution
Re = Constant(3000)
f = Constant((0, 0))

uh, ph = solve_stationary_NS(X, None, Re, f, bc)


niter=50
tollerance=1e-6

for i in range(niter):
  uh_old,ph_old=uh, ph
  uh, ph=solve_stationary_NS(X, uh_old, Re, f, bc)
  
  error=(errornorm(uh, uh_old, 'H1')/norm(uh_old, 'H1')+
         errornorm(ph, ph_old, 'L2')/norm(ph_old, 'L2'))
  print('step {} {:.3e}'. format(i, error))
  
  if error<tollerance:
    break
       
uh.rename('velocity', 'velocity')
ph.rename('pressure', 'pressure')

In [ ]:
whos

In [ ]:
#!/usr/bin/env python

from fenics import *

# 1. mesh generation
nx, ny = 10, 10
mesh = UnitSquareMesh(nx, ny, 'crossed')

# 2. finite element space
degree = 1
V = VectorElement('CG', mesh.ufl_cell(), degree+1)
Q = FiniteElement('CG', mesh.ufl_cell(), degree)

X = FunctionSpace(mesh, MixedElement([V, Q]))

u_boundary = Expression((
        'near(x[1], 1) ? 1.0 : 0.0',
        '0'
    ), degree=0)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [
    DirichletBC(X.sub(0), u_boundary, boundary),
    DirichletBC(X.sub(1), Constant(0), origin, 'pointwise')
]

# 3. problem definition
def solve_linear(X, advection, Re, f, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  a = (inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
  L = dot(f, v) * dx
  if advection:
    a += (dot(grad(u) * advection, v) + dot(grad(advection) * u, v)) * dx
    L += dot(grad(advection) * advection, v) * dx
    
    A = lambda u, p: (grad(u) * advection) + (grad(advection) * u) + grad(p)
    A_SS = lambda u, p: (grad(u) * advection) + (grad(advection) * u) + grad(p)
  
    h = CellDiameter(X.mesh())
    anorm = sqrt(dot(advection, advection))
    tau_K = 0.5 * h / conditional(anorm * h * Re > 12, anorm, 12 / h / Re)
  
    a += tau_K * (dot(A(u, p), A_SS(v, q)) + div(u) * div(v)) * dx
    L += tau_K * (dot(f + grad(advection) * advection, A_SS(v, q))) * dx
  
  x = Function(X)
  solve(a == L, x, bc)
  
  return x.split()

# 4. solution
Re = Constant(300)
f = Constant((0, 0))

uh, ph = solve_linear(X, None, Re, f, bc)

niter = 100
tolerance = 1e-6
for i in range(niter):
  uh_old, ph_old = uh, ph
  uh, ph = solve_linear(X, uh_old, Re, f, bc)
  
  error = (errornorm(uh, uh_old, 'H1') / norm(uh_old, 'H1') +
           errornorm(ph, ph_old, 'L2') / norm(ph_old, 'L2'))
  print('step {}: {:.3e}'.format(i, error))
  if error < tolerance:
    break

uh.rename('velocity', 'velocity')
ph.rename('pressure', 'pressure')

File('velocity.pvd') << uh
File('pressure.pvd') << ph


In [ ]:
clear content